In [1]:
import pandas as pd
import numpy as np
import requests
import pprint
import sidrapy
import warnings 
warnings.simplefilter("ignore")

# Estabelecer a conexão com a API do SIDRA IBGE

In [2]:
API = "https://apisidra.ibge.gov.br/values/t/1612/n3/43/v/allxp/p/last%2024/c81/2713"

Transformar o arquivo json em um Dataframe

In [4]:
req = requests.get(API)
info = req.json()

In [5]:
df = pd.DataFrame(info)
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)

Exluir colunas desnecessárias

In [6]:
df.drop(['Nível Territorial (Código)', 'Nível Territorial',
       'Unidade de Medida (Código)', 'Variável (Código)', 'Ano (Código)', 'Produto das lavouras temporárias (Código)',
       'Unidade de Medida'
       ],
       axis = 1, inplace = True)

Substituir valores não numéricos da coluna Valor e transformá-la em decimal

In [7]:
df.Valor.replace("-", np.nan, inplace = True)
df.Valor.replace("X", np.nan, inplace = True)
df.Valor.replace("...", np.nan, inplace = True)
df.Valor.replace("..", np.nan, inplace = True)

In [8]:
df.Valor = df.Valor.astype(float)

# Seperar os valores em linha e depois fazer o merge em colunas

In [9]:

area_plant = df.loc[df["Variável"] == "Área plantada"]
area_col = df.loc[df["Variável"] == "Área colhida"]
produtiv = df.loc[df["Variável"] == 'Rendimento médio da produção']
prod = df.loc[df["Variável"] == "Quantidade produzida"]
val = df.loc[df["Variável"] == "Valor da produção"]

In [10]:
area_plant.drop("Variável", axis = 1, inplace = True)
area_col.drop("Variável", axis = 1, inplace = True)
produtiv.drop("Variável", axis = 1, inplace = True)
prod.drop("Variável", axis = 1, inplace = True)
val.drop("Variável", axis = 1, inplace = True)

In [93]:
mer = area_plant.merge(area_col, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({
    'Valor_x': 'area_plantada(ha)',
    'Valor_y': 'area_colhida(ha)'
}, axis = 1, inplace = True)

Calcular a diferença entre áreas plantadas e colhidas

In [94]:
mer["delta_area"] = mer['area_colhida(ha)'] - mer['area_plantada(ha)']

In [95]:
mer = mer.merge(prod, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "producao(t)"}, axis = 1, inplace = True)

In [96]:
mer = mer.merge(produtiv, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "produtividade(kg/ha)"}, axis = 1, inplace = True)

In [97]:
mer = mer.merge(val, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "valor_mil(R$)"}, axis = 1, inplace = True)

Calcular o valor médio da saca de soja

In [98]:
mer['valor_saca'] = (mer['valor_mil(R$)'] / ((mer['producao(t)'] / 60))).round(2)

In [99]:
mer.columns

Index(['area_plantada(ha)', 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Ano', 'Produto das lavouras temporárias',
       'area_colhida(ha)', 'delta_area', 'producao(t)', 'produtividade(kg/ha)',
       'valor_mil(R$)', 'valor_saca'],
      dtype='object', name=0)

In [100]:
mer = mer[[ 'Ano', 'Unidade da Federação', 'Produto das lavouras temporárias', 'area_plantada(ha)',
       'area_colhida(ha)', 'delta_area', 'producao(t)', 'produtividade(kg/ha)', 'valor_mil(R$)', 'valor_saca']]

In [101]:
mer.to_parquet("soja_IBGE_RS.parquet", index = False)

In [102]:
mer["var_produtividade"] = mer["produtividade(kg/ha)"].diff()
mer["var_produtividade_%"] = mer["produtividade(kg/ha)"].pct_change()
mer["var_producao(t)"] = mer["produtividade(kg/ha)"].diff()
mer["var_producao(t)_%"] = mer["produtividade(kg/ha)"].pct_change()

In [103]:
mer.head()

,Ano,Unidade da Federação,Produto das lavouras temporárias,area_plantada(ha),area_colhida(ha),delta_area,producao(t),produtividade(kg/ha),valor_mil(R$),valor_saca,var_produtividade,var_produtividade_%,var_producao(t),var_producao(t)_%
0,2000,Rio Grande do Sul,Soja (em grão),3030556.0,3001836.0,-28720.0,4783895.0,1593.0,1385502.0,17.38,NaN,NaN,NaN,NaN
1,2001,Rio Grande do Sul,Soja (em grão),2976498.0,2974513.0,-1985.0,6951830.0,2337.0,2381513.0,20.55,744.0,0.467043,744.0,0.467043
2,2002,Rio Grande do Sul,Soja (em grão),3307252.0,3295342.0,-11910.0,5610518.0,1702.0,2477854.0,26.50,-635.0,-0.271716,-635.0,-0.271716
3,2003,Rio Grande do Sul,Soja (em grão),3591970.0,3591470.0,-500.0,9579297.0,2667.0,5434779.0,34.04,965.0,0.566980,965.0,0.566980
4,2004,Rio Grande do Sul,Soja (em grão),3984337.0,3968530.0,-15807.0,5541714.0,1396.0,3804582.0,41.19,-1271.0,-0.476565,-1271.0,-0.476565


# Importar as variáveis climátivas

# Upload do histórico da variação da temperatura média da superfícia do Oceano Pacífico

Os eventos de El Nino e La nina ocorrem quando há 5 ocorrências de temperaturas 0,5 acima (El Niño) ou abaixo (La Niña) do padrão. Os dados disponíveis nesse dataset

Estudos mostraram que uma condição necessária para o desenvolvimento e persistência da convecção profunda (aumento da nebulosidade e da precipitação) nos Trópicos é que a temperatura da superfície do mar (SST) local seja de 28 °C ou superior. Uma vez que o padrão de convecção profunda é alterado devido a SSTs anômalas, a circulação atmosférica tropical e subtropical se ajusta ao novo padrão de aquecimento tropical, resultando em padrões anômalos de precipitação e temperatura que se estendem muito além da região do Pacífico equatorial. Uma anomalia de SST de +0,5 °C na região do Niño 3.4 é suficiente para atingir esse limite entre o final de março e meados de junho. Durante o restante do ano, uma anomalia de SST maior, chegando a +1,5 °C nos meses de novembro, dezembro e janeiro, é necessária para atingir o limiar que sustenta a convecção profunda persistente nessa região.

In [104]:
enso = pd.read_excel("Temp_media_2010_2025.xlsx")

### Aplicando os ajustes para fazer a fusão entre as bases do IBGE e do NOOA

In [105]:
mer.shape[0]

24

In [106]:
enso.Year.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023, 2024, 2025], dtype=int64)

In [107]:
enso.rename(columns = {"Year": "Ano"}, inplace = True)

In [108]:
mer["Ano"] = mer["Ano"].astype(int)

In [109]:
enso.columns

Index(['Ano', 'Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set',
       'Out', 'Nov', 'Dez', 'Media_mar_jun', 'Media_semestre_1',
       'Media_semestre_2', 'Evento_mar_jun', 'Evento_nov_dez_jan', 'ONI'],
      dtype='object')

In [110]:
mer = mer.merge(enso[["Ano", 'Media_mar_jun', 'Media_semestre_1',
       'Media_semestre_2', 'Evento_mar_jun', 'Evento_nov_dez_jan', 'ONI']], on = "Ano", how = "inner")

In [111]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

mer['Evento_mar_jun_num'] = le.fit_transform(mer['Evento_mar_jun'])
mer['Evento_nov_dez_jan_num'] = le.fit_transform(mer['Evento_nov_dez_jan'])


In [112]:
mer[['delta_area', 'produtividade(kg/ha)', 'ONI']].corr()

,delta_area,produtividade(kg/ha),ONI
delta_area,1.000000,0.591566,0.017745
produtividade(kg/ha),0.591566,1.000000,0.081296
ONI,0.017745,0.081296,1.000000


In [113]:
mer[['delta_area', 'produtividade(kg/ha)', 'Evento_nov_dez_jan_num']].corr()

,delta_area,produtividade(kg/ha),Evento_nov_dez_jan_num
delta_area,1.000000,0.591566,-0.19012
produtividade(kg/ha),0.591566,1.000000,-0.05650
Evento_nov_dez_jan_num,-0.190120,-0.056500,1.00000


In [114]:
mer[['delta_area', 'produtividade(kg/ha)','Evento_mar_jun_num']].corr()

,delta_area,produtividade(kg/ha),Evento_mar_jun_num
delta_area,1.000000,0.591566,-0.077402
produtividade(kg/ha),0.591566,1.000000,-0.352697
Evento_mar_jun_num,-0.077402,-0.352697,1.000000
